### Сбор данных с сайта hh.ru

In [ ]:
#!pip install beautifulsoup4 lxml

Загружались вакансии за последний месяц по ключенвым словам: data science, deep learning, big data и т.д.
Из скачанных данных были отобраны наиболее интересные поля. 

In [1]:
!ls

gathering.py  hh_data_gathering.ipynb  parsers	scrappers  storages


**1. Поиск и загрузка открытых вакансий**

In [5]:
!python3 -m gathering gather

INFO:__main__:Work started
INFO:__main__:Gather process
INFO:scrappers.hh_scrapper:Getting data from hh.ru...
INFO:scrappers.hh_scrapper:text: !"data science" OR !"data scientist" OR !"machine learning" OR !"Deep Learning" OR !"Data Analysis" OR !"Big data" OR Hadoop OR Spark 
INFO:scrappers.hh_scrapper:per_page: 100
INFO:scrappers.hh_scrapper:pages: 10
INFO:scrappers.hh_scrapper:Save 1000 vacancies from hh.ru to the hh_scrapped_data.txt file
INFO:__main__:Work ended


**2. Преобразование загруженных данных в табличный формат**

In [6]:
!python3 -m gathering transform

INFO:__main__:Work started
INFO:__main__:transform
INFO:parsers.hh_parser:Transformed data saved to the hh_data.csv file
INFO:__main__:Work ended


In [7]:
!ls -lh

total 34M
-rw-rw-r-- 1 mariya mariya  926 сен  3 04:54 gathering.py
-rw-rw-r-- 1 mariya mariya 3,3M сен 13 16:20 hh_data.csv
-rw-rw-r-- 1 mariya mariya  23K сен 13 16:17 hh_data_gathering.ipynb
-rw-rw-r-- 1 mariya mariya  30M сен 13 15:51 hh_scrapped_data.txt
drwxrwxr-x 3 mariya mariya 4,0K сен  3 03:09 parsers
drwxrwxr-x 2 mariya mariya 4,0K сен 13 15:21 __pycache__
drwxrwxr-x 3 mariya mariya 4,0K сен  3 04:44 scrappers
drwxrwxr-x 3 mariya mariya 4,0K сен  3 02:21 storages


**3. Краткий анализ загруженных данных**

In [8]:
import pandas as pd
import numpy as np

In [9]:
TABLE_FORMAT_FILE = 'hh_data.csv'

df = pd.read_csv(TABLE_FORMAT_FILE, sep='\t')
print(df.shape)
df.head().T

(1000, 12)


,0,1,2,3,4
address_city,Москва,Москва,Москва,NaN,NaN
description,Our team is building the Data platform to enab...,"В Билайн мы гордимся нашим брендом, как одним ...",Mediascope осуществляет полный цикл работ в об...,Команда Wisebits ищет талантливого и опытного ...,Команда Wisebits ищет талантливого и опытного ...
employment,full,full,full,full,full
experience,between3And6,between1And3,between3And6,between1And3,between1And3
id,27452386,25217817,25214286,27286652,27286653
key_skills,NaN,NaN,NaN,"Python, Big Data, Machine Learning, Математиче...","Python, Big Data, Machine Learning, Математиче..."
name,Chief Data Scientist,Data Scientist,Data Scientist,Data Scientist,Data Scientist
published_at,2018-09-12T17:15:30+0300,2018-09-12T12:59:47+0300,2018-09-12T11:36:58+0300,2018-09-11T11:49:52+0300,2018-09-11T11:49:33+0300
salary_currency,NaN,NaN,NaN,NaN,NaN
salary_from,NaN,NaN,NaN,NaN,NaN


Как часто работодатели указывают зарплатную "вилку"?

In [10]:
ans = (1 - len(df[np.isnan(df['salary_from']) & np.isnan(df['salary_to'])])/df.shape[0])*100
print('{0:.2f} %'.format(ans))

21.80 %


Конвертирование иностранной валюты в рубли:

In [11]:
def convert_salary_from_usd(value):
    return value*68

def convert_salary_from_EUR(value):
    return value*80

df['salary_from_RUB'] = df['salary_from']
df['salary_to_RUB'] = df['salary_to']

filter_usd = df['salary_currency'] == 'USD'
filter_eur = df['salary_currency'] == 'EUR'

df.loc[filter_usd, 'salary_from_RUB'] = df[filter_usd]['salary_from'].apply(convert_salary_from_usd)
df.loc[filter_usd, 'salary_to_RUB'] = df[filter_usd]['salary_to'].apply(convert_salary_from_usd)

df.loc[filter_eur, 'salary_from_RUB'] = df[filter_eur]['salary_from'].apply(convert_salary_from_usd)
df.loc[filter_eur, 'salary_to_RUB'] = df[filter_eur]['salary_to'].apply(convert_salary_from_usd)

Средние нижней и верхней границ зарплаты:

In [12]:
print('Salary from, mean: {} RUB'.format(int(df['salary_from_RUB'].mean())))
print('Salary to, mean: {} RUB'.format(int(df['salary_to_RUB'].mean())))

Salary from, mean: 105857 RUB
Salary to, mean: 175290 RUB


Какой опыт работы чаще всего требуется?

In [13]:
df['experience'].value_counts()

between1And3    495
between3And6    388
noExperience     66
moreThan6        51
Name: experience, dtype: int64

Как меняется зарплата в среднем в зависимости от опыта работы?

In [14]:
df.groupby(['experience'])[['salary_from_RUB', 'salary_to_RUB']].mean()

,salary_from_RUB,salary_to_RUB
experience,,
between1And3,100447.154472,167141.573034
between3And6,114784.000000,179009.756098
moreThan6,196000.000000,258200.000000
noExperience,50088.888889,143250.000000


В каких городах больше всего вакансий?

In [15]:
df['address_city'].value_counts()[:5]

Москва             245
Санкт-Петербург     83
Минск               27
Нижний Новгород     18
Екатеринбург        10
Name: address_city, dtype: int64

Какие ключевые навыки работодатели указывали в вакансии? (посмотрим на топ-20)

In [16]:
import itertools

def get_most_common_skills(skills, n=10):
    key_skills = []

    for i in range(0, len(skills)):
        key_skills.append(skills[i].split(','))

    key_skills = [skill.strip() for skill in list(itertools.chain(*key_skills))]
    
    skills, counts = np.unique(key_skills, return_counts=True)
    return [skills[index] for index in counts.argsort()[::-1][:n]]

In [17]:
get_most_common_skills(df['key_skills'].dropna().values, 20)

['Python',
 'Big Data',
 'SQL',
 'Machine Learning',
 'C++',
 'Java',
 'Linux',
 'Spark',
 'Hadoop',
 'Data Analysis',
 'Deep Learning',
 'Английский язык',
 'Data Mining',
 'Computer Vision',
 'Git',
 'SCALA',
 'JavaScript',
 'Анализ данных',
 'OpenCV',
 'PostgreSQL']

Вывод: Python рулит! :) 